In [1]:
# For the csv file containing all parameters and ranges, create single basin param file based on that

import xarray as xr
import numpy as np
import pandas as pd
import glob, os, sys

sys.path.append('../../MOASMO_support')
from MOASMO_parameters import read_save_load_all_default_parameters


In [2]:
infile_basin_info = f'/glade/work/guoqiang/CTSM_CAMELS/data_mesh_surf/HillslopeHydrology/CAMELS_level1_basin_info.csv'
df_info = pd.read_csv(infile_basin_info)

In [3]:
infile_param = f'../../parameter/CTSM_CAMELS_calibparam_2410.csv'
df_param = pd.read_csv(infile_param)
df_param

,Type,Process,PPE_DailyQ_Rank,Rank2410,Parameter,Default,Lower,Upper,Source,Binding,Method,RangeMethod
0,Hydrology,Soil storage capacity,9,0,watsat_sf,1.000000,0.10000,10.0,Param,NaN,Multiplicative,Absolute
1,Hydrology,Infiltration\r\nSoil storage,1,1,fff,0.500000,0.00100,20.0,Param,NaN,Multiplicative,Absolute
2,Stomatal resistance and photosynthesis,ET (transpiration),2,2,medlynslope,4.840380,0.10000,20.0,Param,NaN,Multiplicative,Absolute
3,Hydrology,Soil water drainage,10,3,hksat_sf,1.000000,0.10000,10.0,Param,NaN,Multiplicative,Absolute
4,Snow Processes,Snow accumulation,24,4,accum_factor,0.100000,0.01000,1.0,Param,NaN,Multiplicative,Absolute
5,Hydrology,Infiltration,0,5,FMAX,0.381669,0.10000,0.9,Surfdata,NaN,Multiplicative,Absolute
6,Hydrology,Infiltration\r\nGroundwater flow,31,6,om_frac_sf,1.000000,0.10000,10.0,Param,NaN,Multiplicative,Absolute
7,Hydrology,Infiltration,23,7,e_ice,6.000000,0.01000,20.0,Param,NaN,Multiplicative,Absolute
8,Hydrology,Soil storage capacity,0,8,zbedrock,NaN,0.10000,10.0,Surfdata,NaN,Multiplicative,Absolute
9,Snow Processes,Snow melt,79,11,n_melt_coef,200.000000,5.00000,800.0,Param,NaN,Multiplicative,Absolute


In [8]:
for basin_num in range(len(df_info)):

    idi = df_info.iloc[basin_num]['hru_id']

    paramfile_new = f'/glade/work/guoqiang/CTSM_CAMELS/data_paramcailb/ParamCalib_same4basins_{idi}.csv'
    
    if os.path.isfile(paramfile_new):
        continue
        
    print('processing', basin_num)
    
    df0 = df_param.copy()
    df0 = df0.rename(columns={'Default':'Default_glob', 'Lower':'Lower_glob', 'Upper':'Upper_glob'})

    # read default param
    path_CTSM_case = f'/glade/work/guoqiang/CTSM_CAMELS/Calib_HH_MOASMO_bigrange/level1_{basin_num}'
    dfnew = read_save_load_all_default_parameters(infile_param, '', path_CTSM_case=path_CTSM_case, savefile=False)

    defa = np.array([np.mean(i) for i in dfnew['Value']])
    df0['Default'] = defa
    if np.any(defa > df0['Upper_glob'].values) or np.any(defa < df0['Lower_glob'].values):
        print('Warning! Bound problem')
        index = (defa > df0['Upper_glob'].values) | (defa < df0['Lower_glob'].values)
        print('Parameters:', df0['Parameter'].values[index])
        print('Parameter defa:', defa[index])
        print('Parameter range:', df0['Lower_glob'].values[index], df0['Upper_glob'].values[index])

        if np.sum(index)>1:
            print('Not only bedrock. stop')
            break
    
    # df0['Lower'] = (defa/df0['Default_glob'].values) * df0['Lower_glob'].values
    # df0['Upper'] = (defa/df0['Default_glob'].values) * df0['Upper_glob'].values
    df0['Lower'] = df0['Lower_glob']
    df0['Upper'] = df0['Upper_glob']

    print('saving to', paramfile_new)
    df0.to_csv(paramfile_new)

processing 0
Load default parameter values from: /all_default_parameters.pkl
saving to /glade/work/guoqiang/CTSM_CAMELS/data_paramcailb/ParamCalib_same4basins_1013500.csv


In [9]:
pd.read_csv('/glade/work/guoqiang/CTSM_CAMELS/data_paramcailb/ParamCalib_same4basins_1013500.csv')

,Unnamed: 0,Type,Process,PPE_DailyQ_Rank,Rank2410,Parameter,Default_glob,Lower_glob,Upper_glob,Source,Binding,Method,RangeMethod,Default,Lower,Upper
0,0,Hydrology,Soil storage capacity,9,0,watsat_sf,1.000000,0.10000,10.0,Param,NaN,Multiplicative,Absolute,1.000000,0.10000,10.0
1,1,Hydrology,Infiltration\r\nSoil storage,1,1,fff,0.500000,0.00100,20.0,Param,NaN,Multiplicative,Absolute,0.500000,0.00100,20.0
2,2,Stomatal resistance and photosynthesis,ET (transpiration),2,2,medlynslope,4.840380,0.10000,20.0,Param,NaN,Multiplicative,Absolute,4.840380,0.10000,20.0
3,3,Hydrology,Soil water drainage,10,3,hksat_sf,1.000000,0.10000,10.0,Param,NaN,Multiplicative,Absolute,1.000000,0.10000,10.0
4,4,Snow Processes,Snow accumulation,24,4,accum_factor,0.100000,0.01000,1.0,Param,NaN,Multiplicative,Absolute,0.100000,0.01000,1.0
5,5,Hydrology,Infiltration,0,5,FMAX,0.381669,0.10000,0.9,Surfdata,NaN,Multiplicative,Absolute,0.418641,0.10000,0.9
6,6,Hydrology,Infiltration\r\nGroundwater flow,31,6,om_frac_sf,1.000000,0.10000,10.0,Param,NaN,Multiplicative,Absolute,1.000000,0.10000,10.0
7,7,Hydrology,Infiltration,23,7,e_ice,6.000000,0.01000,20.0,Param,NaN,Multiplicative,Absolute,6.000000,0.01000,20.0
8,8,Hydrology,Soil storage capacity,0,8,zbedrock,NaN,0.10000,10.0,Surfdata,NaN,Multiplicative,Absolute,5.689264,0.10000,10.0
9,9,Snow Processes,Snow melt,79,11,n_melt_coef,200.000000,5.00000,800.0,Param,NaN,Multiplicative,Absolute,200.000000,5.00000,800.0
